In [ ]:
# import numpy as np
# import nibabel as nib
# import matplotlib.pyplot as plt
# import os 
# opj = os.path.join


%load_ext autoreload
%autoreload 2
import scipy.io
import numpy as np  
import scipy.io
import matplotlib.pyplot as plt
import os
import linescanning.utils as lsutils
import linescanning.plotting as lsplt
opj = os.path.join

from amb_scripts.load_saved_info import *
from amb_scripts.plot_functions import *
# from amb_scripts.plot_shifts import *

# from prfpy.rf import csf_exponential
from amb_scripts.plot_csf import *


In [ ]:
# prf_obj 
# prf_obj = Prf1T1M('sub-01', 'LE', 'norm')

In [ ]:
a = prf_obj.return_vx_mask(th={'min-rsq':.1})

In [ ]:
# Load the dm stuff
from amb_scripts.load_saved_info import *
csf_dm = amb_load_dm(['sf_vect', 'c_vect'])
sf_vect = csf_dm['sf_vect']
c_vect = csf_dm['c_vect']
csf_seq = np.squeeze(scipy.io.loadmat('./csf_image_seq.mat')['csf_image_seq']) - 1 # -1 because matlab numbering
csf_seqtiming = np.squeeze(scipy.io.loadmat('./csf_seqtiming.mat')['stim_seqtiming']) 
csf_images = scipy.io.loadmat('./csf_images270.mat')['csf_images']

In [ ]:
roi_fit = 'all'
fit_stage = 'iter'
G_plot = {}
G_plot['sub-01'] = AmbPlotter('sub-01', roi_fit=roi_fit, fit_stage=fit_stage)
G_plot['sub-02'] = AmbPlotter('sub-02', roi_fit=roi_fit, fit_stage=fit_stage)

In [ ]:
# SOMETHING TO SET IT UP NICE? LINESCANNING LAZYPLOT..
# sub = 'sub-02'
# idx = 0
# fig = plt.figure(constrained_layout=True, figsize=(15,5))
# ax = fig.add_subplot()
# lsplt.LazyPlot(
#     G_plot[sub].real_tc['CSF']['LE'][idx,0:50],
#     color='k', 
#     labels='CSF-LE', 
#     add_hline='default',
#     x_label='time',
#     axs=ax,
    
#     # title=set_title,
#     # xkcd=True,
#     # font_size=font_size,
#     # line_width=LP_lw,
#     # markers=LP_marker,
#     # **kwargs,
#     )


# MODEL EXPLANATION

In [ ]:
# Start simple -> The gratings + the 

# TIME SERIES 

In [ ]:
sub='sub-02'
eye = 'LE'
idx = np.where(G_plot['sub-02'].return_vx_mask({
    'CSF-LE-min-rsq':.4,
    'CSF-LE-bound-bold_baseline':[-.5,.5]}))[0][1]


save_name = f'./csf_tc_{sub}-{eye}-vx{idx}'
if not os.path.exists(save_name):
    os.mkdir(save_name)
i_0 = 0
i_m1 = 214
i_step = 1
i_fig = 0
for i in range(i_0,i_m1,i_step):
    fig = G_plot[sub].csf_tc_plotV3(eye, idx, i)

    fig.savefig(f"{save_name}/file{i_fig:03}.png")
    plt.close(fig)
    print(i_fig)
    i_fig += 1

# ffmpeg_dir = "/home/**your_name**/.conda/envs/**env_name**/bin/ffmpeg"
ffmpeg_dir = '~/.conda/envs/dnsim_as/bin/ffmpeg'
# Select the directory with all of the images, and file name pattern
# >> -r 15 controls frame rate, can be faster or slower 
os.system(f"{ffmpeg_dir} -r 5 -i {save_name}/file%03d.png -codec mpeg4 -y {save_name}_movie.mp4")
os.system(f"rm -R {save_name}")
    

In [ ]:
sub='sub-02'
eye = 'LE'
idx = np.where(G_plot['sub-02'].return_vx_mask({
    'CSF-LE-min-rsq':.4,
    'CSF-LE-bound-bold_baseline':[-.5,.5]}))[0][1]


save_name = f'./csf_tc_{sub}-{eye}-vx{idx}'
# if not os.path.exists(save_name):
#     os.mkdir(save_name)
i_0 = 200
i_m1 = 214
i_step = 1
i_fig = 0
for i in range(i_0,i_m1,i_step):
    fig = G_plot[sub].csf_tc_plotV3(eye, idx, i)

    # fig.savefig(f"{save_name}/file{i_fig:03}.png")
    # plt.close(fig)
    print(i_fig)
    i_fig += 1


In [ ]:
def grate_texture(sf=1, con=1, n_pix=200):
    n_deg = 5
    x_grid,y_grid = np.meshgrid(np.linspace(-n_deg,n_deg,n_pix),np.linspace(-n_deg,n_deg,n_pix))
    ecc = np.sqrt(x_grid**2 + y_grid**2)
    grate = np.cos(x_grid*sf) * con
    grate[ecc>n_deg] = 0
    # plt.imshow(grate, vmin=-1, vmax=1)
    return grate

CSF_stim=amb_load_prfpy_stim('CSF')
def make_csf_img_graph(t_pt, CSF_stim):
    fig, ax = plt.subplots(1,3, gridspec_kw={'width_ratios': [2,2,1]})
    fig.set_size_inches(12,5)
    grate = grate_texture(sf=CSF_stim.SF_seq[t_pt], con=CSF_stim.CON_seq[t_pt]/100)
    ax[0].imshow(grate, vmin=-1, vmax=1, cmap='Greys')
    ax[0].axis('off')
    ax[0].set_title(f'SF={CSF_stim.SF_seq[t_pt]:.3f}, C={CSF_stim.CON_seq[t_pt]:.3f}')

    ax[1].set_yscale('log')
    ax[1].set_xscale('log')

    ax[1].scatter(CSF_stim.SF_seq[0:t_pt], 100/CSF_stim.CON_seq[0:t_pt], c='k', s=100)
    if sf_vect[t_pt]==0:
        ax[1].text(.5, .5, 'BASELINE',
                horizontalalignment='center',
                verticalalignment='top',
                backgroundcolor='1',
                transform=ax[1].transAxes)            
    else:        
        ax[1].scatter(CSF_stim.SF_seq[t_pt], 100/CSF_stim.CON_seq[t_pt], marker='*', c='g', s=500)

    ax[1].set_xlim(.25,20)
    ax[1].set_ylim(1,500)
    ax[1].set_aspect('equal')
    ax[1].set_xlabel('SF')
    ax[1].set_ylabel('100/Contrast')

    ax[2].imshow(CSF_stim.design_matrix[:,:,t_pt], vmin=0, vmax=1)
    ax[2].axis('off')
    ax[2].set_title('DM space')
    fig.set_tight_layout('tight')
    update_fig_fontsize(fig, 20)

    return fig


In [ ]:

save_name = './csf_img_graph'
if not os.path.exists(save_name):
    os.mkdir(save_name)
i_0 = 0
i_m1 = sf_vect.shape[0]
i_step = 1
i_fig = 0
for i in range(i_0,i_m1,i_step):

    fig = make_csf_img_graph(i, CSF_stim)
    fig.savefig(f"{save_name}/file{i_fig:03}.png")
    plt.close(fig)
    print(i_fig)
    i_fig += 1

# ffmpeg_dir = "/home/**your_name**/.conda/envs/**env_name**/bin/ffmpeg"
ffmpeg_dir = '~/.conda/envs/dnsim_as/bin/ffmpeg'
# Select the directory with all of the images, and file name pattern
# >> -r 15 controls frame rate, can be faster or slower 
os.system(f"{ffmpeg_dir} -r 5 -i {save_name}/file%03d.png -codec mpeg4 -y {save_name}_movie.mp4")
os.system(f"rm -R {save_name}")

In [ ]:



# i_0 = 200
# i_m1 = 214#sf_vect.shape[0]
# i_step = 1
# i_fig = 0
# for i in range(i_0,i_m1,i_step):

#     fig = make_csf_img_graph(i, CSF_stim)
#     plt.figure()
#     i_fig += 1



In [ ]:

# save_name = './csf_img_graph'
# if not os.path.exists(save_name):
#     os.mkdir(save_name)
i_0 = 0
i_m1 = sf_vect.shape[0]
i_step = 1
i_fig = 0
for i in range(i_0,i_m1,i_step):
    fig, ax = plt.subplots(1,2)
    fig.set_size_inches(10,5)
    grate = grate_texture(sf=sf_vect[i], con=c_vect[i]/100)
    ax[0].imshow(grate, vmin=-1, vmax=1, cmap='Greys')
    ax[0].axis('off')

    ax[1].set_yscale('log')
    ax[1].set_xscale('log')

    ax[1].scatter(sf_vect[i_0:i], c_vect[i_0:i], c='k')
    ax[1].scatter(sf_vect[i], c_vect[i], c='r')
    ax[1].set_ylim(10**-1,10**2)
    ax[1].set_xlim(.1,20)
    ax[1].set_xlabel('SF')
    ax[1].set_ylabel('Contrast')
    fig.set_tight_layout('tight')
    update_fig_fontsize(fig, 20)

    fig.savefig(f"{save_name}/file{i_fig:03}.png")
    plt.close(fig)
    print(i_fig)
    i_fig += 1

# ffmpeg_dir = "/home/**your_name**/.conda/envs/**env_name**/bin/ffmpeg"
ffmpeg_dir = '~/.conda/envs/dnsim_as/bin/ffmpeg'
# Select the directory with all of the images, and file name pattern
# >> -r 15 controls frame rate, can be faster or slower 
os.system(f"{ffmpeg_dir} -r 4 -i {save_name}/file%03d.png -codec mpeg4 -y {save_name}_movie.mp4")
os.system(f"rm -R {save_name}")

In [ ]:
sub = 'sub-02'
CSF_stim = amb_load_prfpy_stim('CSF')
# SFs = np.linspace(0.5,18,30)
# log_SFs = np.log10(SFs)
# S_curve = np.squeeze(get_csf_curves(
#     log_SFs=log_SFs,
#     width_r=2.5,
#     SFp=1.12,
#     CSp=68.97,
#     width_l=0.448))
eye='LE'
min_rsq = 0.1
vx_mask = G_plot[sub].return_vx_mask({f'CSF-{eye}-min-rsq':min_rsq})
ps_to_vary = ['width_r', 'SFp', 'CSp']
width_l = 0.448
# [1] Find the p_range, and weighted mean (used for ...)
p_range = {}
p_wmean = {}
for i in ps_to_vary:
    this_p = G_plot[sub].return_th_param('CSF', eye, i, vx_mask)
    q1 = np.percentile(this_p, 25)
    q2 = np.percentile(this_p, 75)    
    p_range[i] = np.linspace(q1,q2,5)

    p_wmean[i] = G_plot[sub].return_w_mean('CSF', eye, i, vx_mask)


In [ ]:
cmap_list = ['Reds_r', 'Blues', 'Greens']
for i_cmap,this_p in enumerate(ps_to_vary):
    fig, ax = plt.subplots(1)
    fig.set_size_inches(10,15)
    # Set up axes
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.scatter(CSF_stim.SF_seq, 100/CSF_stim.CON_seq, c='k')
    ax.set_xlabel('SF')
    ax.set_ylabel('100/Contrast')
    # ax[i].plot(SFs, S_curve, lw=5)
    y_lim = (1, 500)
    x_lim = (.25,20)
    ax.set_ylim(y_lim)
    ax.set_xlim(x_lim)
    ax.set_aspect('equal')

    # AX [1] Vary width_r:
    if '_r' in cmap_list[i_cmap]:
        col = plt.cm.__dict__[cmap_list[i_cmap]](np.linspace(0,.5,5))
    else:
        col = plt.cm.__dict__[cmap_list[i_cmap]](np.linspace(.5,1,5))
    for i,i_pval in enumerate(p_range[this_p]):
        if this_p=='width_r':
            this_width_r = i_pval
        else:
            this_width_r = p_wmean['width_r']
        if this_p=='SFp':
            this_SFp = i_pval
        else:
            this_SFp = p_wmean['SFp']

        if this_p=='CSp':
            this_CSp = i_pval
        else:
            this_CSp = p_wmean['CSp']
        
        S_curve = np.squeeze(get_csf_curves(
            log_SFs=CSF_stim.log_SFs,
            width_r=this_width_r,
            SFp=this_SFp,
            CSp=this_CSp,
            width_l=0.448))    
        ax.plot(CSF_stim.SFs, S_curve, label=f'{this_p} {i_pval:.2f}', lw=5, c=col[i])
    # ax[0].legend()
    param_text = ''
    for p4txt in ps_to_vary:
        if p4txt==this_p:
            continue
        param_text += f'{p4txt:>7}={p_wmean[p4txt]:.2f}\n'
    ax.legend(bbox_to_anchor=(1.05, 1.05), prop={'size':15,'family': 'monospace'})
    ax.text(1.3,.5, param_text,                     
            horizontalalignment='center',
            verticalalignment='top',
            backgroundcolor='1',
            transform=ax.transAxes) 

    fig.set_tight_layout('tight')
    update_fig_fontsize(fig, 20)
    fig.savefig(f'{this_p}.png')




In [ ]:
min_rsq = 0.1
eye = 'LE'
vx_mask = G_plot[sub].return_vx_mask({'CSF-LE-min-rsq':min_rsq})
for i in G_plot[sub].p_labels['CSF'].keys():
    
    this_wp = G_plot[sub].return_w_mean('CSF', eye, i, vx_mask)
    print(f'{i} = {this_wp}')

In [ ]:
cmap_dict = {
    'width_r' : 'Reds_r',       
    'SFp' : 'Blues'     ,
    'CSp' : 'Greens',
    'pol' : None   ,   
    'ecc' : 'magma' ,    
    'rsq' : 'cool'   ,     
}
for sub in ['01', '02']:
    for eye in ['LE', 'RE']:
        for param in ['width_r', 'SFp', 'CSp', 'pol', 'ecc', 'rsq']:
            if (param=='pol') | (param=='ecc'):
                model = 'gauss'
                task = f'pRF{eye}'
            else:
                model = 'CSF'
                task = f'CSF{eye}'
            
            fview_cmd = f'python ../amb_scripts/data_prf_fits_scripts/freesurfer_pol_plot.py -s {sub} -t {task} --param {param}  --model {model} --scr_shot --cmap {cmap_dict[param]}'

            os.system(fview_cmd)
            input()
            


In [ ]:
for sub in ['01', '02']:
    for eye in ['LE', 'RE']:
        for param in ['width_r', 'SFp', 'CSp', 'pol', 'ecc', 'rsq']:
            if (param=='pol') | (param=='ecc'):
                model = 'gauss'
            else:
                model = 'CSF'
            
            fview_cmd = f'python ../amb_scripts/data_prf_fits_scripts/freesurfer_pol_plotV2.py -s {sub} -e {eye} --param {param} --model {model} --scr_shot'
            # print(fview_cmd)
            os.system(fview_cmd)
            


In [ ]:
for sub in ['01', '02']:
    for eye in ['LE', 'RE']:
        for param in ['sfmax']:
            model = 'CSF'
            
            fview_cmd = f'python ../amb_scripts/data_prf_fits_scripts/freesurfer_pol_plotV2.py -s {sub} -e {eye} --param {param} --model {model} --scr_shot'
            # print(fview_cmd)
            os.system(fview_cmd)
            


In [ ]:
for sub in ['01', '02']:
    for eye in ['LE', 'RE']:
        for param in ['sfmax']:
            model = 'CSF'            
            fview_cmd = f'python ../amb_scripts/data_prf_fits_scripts/freesurfer_pol_plotV2.py -s {sub} -e {eye} --param {param} --model {model} --scr_shot'

            os.system(fview_cmd)
            a = input()
            if a=='q':
                sys.exit()


In [ ]:
prf_obj = Prf1T1M('sub-01', 'LE', 'CSF')

In [ ]:
np.log10(-1)

In [ ]:
prf_obj.pd_params['SFp']

In [ ]:
roi_fit = 'all'
fit_stage = 'iter'
G_plot = {}
G_plot['sub-01'] = AmbPlotter('sub-01', roi_fit=roi_fit, fit_stage=fit_stage)
G_plot['sub-02'] = AmbPlotter('sub-02', roi_fit=roi_fit, fit_stage=fit_stage)

In [ ]:
# Histograms LE vs RE
import seaborn as sns
import pandas as pd
model = 'CSF'
min_rsq = 0.1
for sub in ['sub-01', 'sub-02']:
    for param in ['width_r', 'SFp', 'CSp', 'rsq',]:

        fig, ax = plt.subplots(1)
        fig.set_size_inches(10,10)
        this_param_dict = {}        
        this_param_dict[param] = []
        this_param_dict['eye'] = []
        for eye in ['LE', 'RE']:
            vx_mask = G_plot[sub].return_vx_mask(th_dict={f'{model}-{eye}-min-rsq': min_rsq})                
            this_param_dict[param].append(G_plot[sub].return_th_param(model, eye, param, vx_mask).to_numpy())                
            this_param_dict['eye'].append([eye]*len(G_plot[sub].return_th_param(model, eye, param, vx_mask).to_numpy()))
        for key in this_param_dict.keys():
            this_param_dict[key] = np.concatenate(this_param_dict[key])
        this_param_pd = pd.DataFrame(this_param_dict)
        sns.violinplot(data=this_param_pd,y=param,hue='eye',split=True)# color=G_plot[sub].plot_cols[eye], facealpha=.5)
        # sns.boxplot(this_param)
        # G_plot[sub].hist_generic(axs=ax, vx_mask=vx_mask, param=f'{model}-{eye}-{param}', title=sub,)

        # ax.plot((0,1), (0,1), 'k')
        update_fig_fontsize(fig, 20)        
    

In [ ]:
# Histograms LE vs RE
plot_cols = get_plot_cols()
min_rsq = 0.1
max_ecc = 5
roi = 'V1_exvivo'

model_list = ['CSF', 'gauss']

m_param_list = {
    'CSF' : ['sfmax', 'width_r', 'SFp', 'CSp', 'rsq'],
    'gauss' : ['ecc', 'a_sigma', 'rsq'],
}

n_bins = 20
m_param_bins = {
    'sfmax' : np.linspace(1,30,n_bins),
    'width_r' : np.linspace(0,3,n_bins),
    'SFp' : np.linspace(0,6,n_bins),
    'CSp' : np.linspace(0,200,n_bins),
    'rsq' : np.linspace(0,1,n_bins),
    'ecc' : np.linspace(0,5,n_bins),
    'a_sigma' : np.linspace(0,5,n_bins),

}

for sub in ['sub-01', 'sub-02']:
    for model in model_list:
        for param in m_param_list[model]:
            fig,(ax_box,ax_hist)=plt.subplots(nrows=2,sharex=True, gridspec_kw={'height_ratios':(.25,.75)},figsize=(10,10)) 
            roi_mask = amb_load_roi(sub, roi)
            this_p = []
            for eye in ['LE', 'RE']:
                vx_mask = G_plot[sub].return_vx_mask(th_dict={f'{model}-{eye}-min-rsq': min_rsq,f'{model}-{eye}-max-ecc': max_ecc, 'roi':roi_mask})                
                this_p.append(G_plot[sub].return_th_param(model, eye, param, vx_mask))
                G_plot[sub].hist_generic(axs=ax_hist, vx_mask=vx_mask, param=f'{model}-{eye}-{param}', bins=m_param_bins[param])
            
            bplot = ax_box.boxplot(this_p, showfliers=False, vert=False, widths=.5, patch_artist=True)
            bplot_cols = [plot_cols['LE'], plot_cols['RE']]
            for patch, color in zip(bplot['boxes'], bplot_cols):
                patch.set_facecolor(color)        

            # ax_box.set_xlim(0,20)
            # # ax.plot((0,1), (0,1), 'k')
            fig.suptitle(f"{roi}-{sub}-{model}-{param}")
            fig.tight_layout()
            update_fig_fontsize(fig, 20)
            fig.savefig(f'{roi}-{sub}-{model}-{param}.png')

        

In [ ]:
type(None)

In [ ]:
a = np.where(G_plot['sub-01'].return_vx_mask({'CSF-LE-min-rsq':.1, 'CSF-LE-min-sfmax':99999999}))[0]

np.where(G_plot['sub-01'].pd_params['CSF']['LE']['sfmax']==np.inf)